In [1]:
!pip install langchain_nvidia_ai_endpoints

In [4]:
!pip install langchain_community

In [2]:
!pip install langchain-community langchain-text-splitters faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 8.0 MB/s eta 0:00:00


In [1]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings, ChatNVIDIA

In [2]:
import os

In [3]:
from langchain_community.document_loaders import WebBaseLoader

In [4]:
loader = WebBaseLoader("https://medium.com/@artgor/paper-review-personalized-audiobook-recommendations-at-spotify-through-graph-neural-networks-bd24cf0dd198")

In [5]:
docs=loader.load()

In [6]:
from google.colab import userdata
os.environ['NVIDIA_API_KEY']=userdata.get('google_gemma')

In [7]:
embeddings= NVIDIAEmbeddings()

In [8]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [9]:
text_splitter= RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)

In [10]:
document=text_splitter.split_documents(docs)

In [11]:
document[0]

Document(page_content='Paper Review: Personalized Audiobook Recommendations at Spotify Through Graph Neural Networks | by Andrew Lukyanenko | Mar, 2024 | MediumOpen in appSign upSign inWriteSign upSign inPaper ReviewPaper Review: Personalized Audiobook Recommendations at Spotify Through Graph Neural NetworksRecommendation system for Audiobooks using GNN!Andrew Lukyanenko·Follow6 min read·3 days ago--ListenSharePaper linkSpotify has recently expanded its offerings to include audiobooks, presenting challenges in', metadata={'source': 'https://medium.com/@artgor/paper-review-personalized-audiobook-recommendations-at-spotify-through-graph-neural-networks-bd24cf0dd198', 'title': 'Paper Review: Personalized Audiobook Recommendations at Spotify Through Graph Neural Networks | by Andrew Lukyanenko | Mar, 2024 | Medium', 'description': 'Spotify has recently expanded its offerings to include audiobooks, presenting challenges in personalized recommendation due to the inability to skim audiobooks 

In [12]:
vector=FAISS.from_documents(document,embeddings)

In [13]:
reteiver=vector.as_retriever()

In [14]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [15]:
model = ChatNVIDIA(model="mistral_7b")

In [16]:
hyde_template = """Even if you do not know the full answer, generate a one-paragraph hypothetical answer to the below question:

{question}"""
hyde_prompt = ChatPromptTemplate.from_template(hyde_template)
hyde_query_transformer = hyde_prompt | model | StrOutputParser()

In [17]:
hyde_prompt

ChatPromptTemplate(input_variables=['question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='Even if you do not know the full answer, generate a one-paragraph hypothetical answer to the below question:\n\n{question}'))])

In [18]:
from langchain_core.runnables import chain

In [19]:
@chain
def hyde_ret(question):
  hypo_document= hyde_query_transformer.invoke({"question":question})
  return retriever.invoke(hypo_document)

In [20]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
answer_chain = prompt | model | StrOutputParser()

In [21]:
@chain
def final_chain(question):
    documents = hyde_query_transformer.invoke(question)
    for s in answer_chain.stream({"question": question, "context": documents}):
        yield s

In [22]:
for s in final_chain.stream("what is the spotify audiobook recommendation"):
    print(s, end="")

The Spotify audiobook recommendation system suggests titles based on your listening history and preferences in the Genres, Authors, and Narrators sections of your Spotify account. It uses this information to recommend audiobooks that align with your interests. However, it's important to note that the recommendation system is not perfect, and you may receive suggestions that don't entirely match your preferences. To refine your recommendations, you can provide feedback on the suggested titles, rate or save the ones you enjoy, and explore new genres or authors.